In [4]:
import numpy as np
import sklearn as skl
import pandas as pd
import sklearn.preprocessing
from sklearn import linear_model
from sklearn.metrics import f1_score
from scipy import special
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import keras
from keras.layers import Dense, BatchNormalization
from keras.models import Sequential
from tensorflow.python.client import device_lib

np.random.seed(69) #fixing seed for reproducability

In [5]:
#print(device_lib.list_local_devices())

df_train = pd.read_csv("train.csv")
df_splitt = df_train['Sequence'].apply(lambda x: pd.Series(list(x))) #splitts the 4 mutators into individual columns.

#df_letters = pd.concat([df_train['Sequence'].apply(lambda x: pd.Series(list(x))), df_train['Active']],axis=1)

In [6]:
df_test = pd.read_csv("test.csv")
df_test_s = df_test['Sequence'].apply(lambda x: pd.Series(list(x)))

In [7]:
# define universe of possible input values
mutators = 'ACDEFGHIKLMNPQRSTUVWY' #abcdefghijklmnopqrstuvwxyz
#bjoqxz <-- these letters are missing

# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(mutators))
int_to_char = dict((i, c) for i, c in enumerate(mutators))

# integer encode input data
#integer_encoded = [char_to_int[item] for item in df_letters[0]]
#print(integer_encoded)

In [8]:
df = np.zeros((112000, 21*4))
#Possibly the worst fucking way we could have done this. But fuckem.
for i in range(0,4):
    for j in range(0, 112000):
        temp = char_to_int[df_splitt.loc[j,i]]
        df[j, i*21 + temp] = 1      #binary representation of the letters
        #df[j, i] = temp             #letter as float (0,1]

In [9]:
#Do the same with the test dataframe
df_cock = np.zeros((48000, 21*4))

for i in range(0,4):
    for j in range(0, 48000):
        temp = char_to_int[df_test_s.loc[j,i]]
        df_cock[j, i*21 + temp] = 1
        #df_cock[j, i] = temp

In [10]:
clf = skl.linear_model.RidgeCV(alphas=[0.01, 0.01, 0.1, 1, 10, 100, 1000, 2000,5000], cv=15)

labels = df_train['Active'].to_numpy()

#predict cocks:
#clf.fit(df,labels)
print('Done with fitting')
#predict_train = clf.predict(df)
#print('Done with prediction and error is: ', np.sqrt(np.mean((predict_train-labels)**2)))
#predict_cancer = clf.predict(df_cock)
#predict_cancer = np.where(predict_cancer > 0.5, 1, 0)

Done with fitting


In [11]:
#clf = skl.svm.LinearSVC(dual=False, class_weight='balanced')

#labels = df_train['Active'].to_numpy()

#predict cocks:
#clf.fit(df,labels)
print('Done with fitting')
#predict_train = clf.predict(df)
#print('Done with prediction and error is: ', np.sqrt(np.mean((predict_train-labels)**2)))
#predict_cancer = clf.predict(df_cock)
#predict_cancer = np.where(predict_cancer > 0.5, 1, 0)

Done with fitting


In [12]:
#np.savetxt('submission_sklearn.csv', predict_cancer, delimiter=',', fmt='%i')

In [13]:
#lets norm this shit (this is only necessary if we use the 4D representation
#df_nn = (df+1)/21
#df_cock_nn = (df_cock+1)/21

#for the binary version use this:
df_nn = df
df_cock_nn = df_cock

#print(df_nn.max(), df_nn.min())
#print(labels.max(), labels.min())
#print(df_cock_nn.max(), df_cock_nn.min())

In [18]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when val_loss is no longer improving
        monitor="loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-3,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
    )
]

In [19]:
#lets construct the neural net:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(128, activation='relu', input_dim=84))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(1, activation='sigmoid'))
#neuralNetwork.add(BatchNormalization())
neuralNetwork.compile(loss = keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
#neuralNetwork.fit(df_nn, labels, epochs=50, batch_size=1024, verbose=1, callbacks= callbacks)

#version without loss monitoring
neuralNetwork.fit(df_nn, labels, epochs=50, batch_size=1024, verbose=1, callbacks= callbacks)


#evaluation of the network prediction
predict_cancer_nn = neuralNetwork.predict(df_cock_nn)
predict_train_nn = neuralNetwork.predict(df_nn)
print(predict_cancer_nn)
print(predict_train_nn)
predict_cancer_nn = (predict_cancer_nn >= 0.5)

Epoch 1/50
110/110 [==============================] - 2s 11ms/step - loss: 0.5701 - accuracy: 0.7339
Epoch 2/50
110/110 [==============================] - 1s 9ms/step - loss: 0.0978 - accuracy: 0.9839
Epoch 3/50
110/110 [==============================] - 1s 9ms/step - loss: 0.0378 - accuracy: 0.9901
Epoch 4/50
110/110 [==============================] - 1s 9ms/step - loss: 0.0241 - accuracy: 0.9930
Epoch 5/50
110/110 [==============================] - 1s 9ms/step - loss: 0.0176 - accuracy: 0.9943
Epoch 6/50
110/110 [==============================] - 1s 9ms/step - loss: 0.0132 - accuracy: 0.9961
Epoch 7/50
110/110 [==============================] - 1s 8ms/step - loss: 0.0116 - accuracy: 0.9965
Epoch 8/50
110/110 [==============================] - 1s 8ms/step - loss: 0.0094 - accuracy: 0.9972
Epoch 9/50
110/110 [==============================] - 1s 8ms/step - loss: 0.0064 - accuracy: 0.9982
Epoch 10/50
110/110 [==============================] - 1s 8ms/step - loss: 0.0061 - accuracy: 0.998

In [16]:
print(predict_train_nn.max())
print(predict_train_nn.min())

print("actual labels:", np.sum(labels))
print("predicted labels:", np.sum(predict_train_nn >0.5))
print("the f1 score is:", f1_score(labels.astype('bool'), (predict_train_nn >=0.5).astype('bool')))

1.0
6.60469e-06
actual labels: 4213
predicted labels: 3715
the f1 score is: 0.9308779011099899


In [17]:
predict_cancer_nn = pd.DataFrame(predict_cancer_nn.astype('int'))
predict_cancer_nn.to_csv('predictions.csv', header = False, index = False)
